In [3]:
import numpy as np
import matplotlib.pyplot as plt
import Flow_Solutions_Class as FS
from scipy.special import lambertw
from scipy.optimize import brentq
from scipy.interpolate import interp1d
from scipy.integrate import quad
%matplotlib inline

ModuleNotFoundError: No module named 'Flow_solutions_Class'

In [ ]:


G = 6.67e-8
kappa_p = 1e-2
kappa_op = 4e-3
mmw = 2.35
mh = 1.67e-24
kb =1.38e-16

rearth = 6.371e8
mearth = 5.97e27

Teq = 1000. 
cs_eq = np.sqrt(kb*Teq/mh/mmw)
Fbol = 4.*5.6705e-5*Teq**4.


FEUV = 450. # energy flux
FEUV_photon = FEUV / (20.*1.6e-12) # photon flux

#r_p = 5.3 * 6.371e8
#Mp = 20.*5.97e27
#g = G * Mp / r_p**2.

#Rs_iso = G*Mp/2./cs_eq**2.

#rho_photo = g / (kappa_p * cs_eq**2.)

In [ ]:
def compute_mdot(cs,REUV,Mp,sigma_EUV,Mdot_want):
    
    #### this routine computes the mass-loss rates for different sound-speeds
    #### assuming that the optical depth to EUV is 1
    
    ## compute sonic point
    
    RS_flow = G * Mp / (2.*cs**2.)
    
    if (RS_flow >= REUV):
        ## sonic point exists inside domain
        r = np.logspace(np.log10(REUV),np.log10(max(5*RS_flow,5*REUV)),250)
        
        u = FS.get_parker_wind(r,cs,RS_flow)
        
        rho = (RS_flow/r)**2.*(cs/u) # density such that density at sonic point is 1
        
        ## now scale such that optical depth to EUV is 1
        
        tau = np.fabs(np.trapz(rho[::-1],r[::-1]))
        
        rho_s = 1./((sigma_EUV/(mmw*mh/2.))*tau)
    
        rho = rho*rho_s
        
        Mdot = 4*np.pi*REUV**2.*rho[0]*u[0]
        
        print(rho[0])
        
    else:
        ## force launch at sound-speed
        
        ## sonic point exists inside domain
        r = np.logspace(np.log10(REUV),np.log10(max(5*RS_flow,5*REUV)),250)
        
        constant = (1. - 4.*np.log(REUV/RS_flow) - 4.*(RS_flow/REUV)-1e-13) # need small safety factor
        #constant = -3.
        
        u = FS.get_parker_wind_const(r,cs,RS_flow,constant)
        
        rho = (RS_flow/r)**2.*(cs/u) # density such that density at sonic point is 1
        
        ## now scale such that optical depth to EUV is 1
        
        tau = np.fabs(np.trapz(rho[::-1],r[::-1]))
        
        rho_s = 1./((sigma_EUV/(mmw*mh/2.))*tau)
    
        rho = rho*rho_s
        
        Mdot = 4*np.pi*REUV**2.*rho[0]*u[0]
        
    return (Mdot-Mdot_want)/(Mdot_want+1.)
   

def compute_sound_speed(REUV,Mp,sigma_EUV,eff):
    
    lower_bound_initial = 2e5
    
    Mdot_EL = eff* np.pi * REUV**3. / (4.*G*Mp) * FEUV
    f1 = compute_mdot(lower_bound_initial,REUV,Mp,sigma_EUV,Mdot_EL)
    
    print (Mdot_EL)
    
    #f2 = compute_mdot(1e12,REUV,Mp,sigma_EUV,Mdot_EL)
    
    if (f1 < 0):
        cs_use = brentq(compute_mdot,lower_bound_initial,1e13,args=(REUV,Mp,sigma_EUV,Mdot_EL))
    else:
        print ("reducing lower_bound")
        lower_bound = lower_bound_initial
        while f1 > 0:
            lower_bound = 0.95*lower_bound
            f1 = compute_mdot(lower_bound,REUV,Mp,sigma_EUV,Mdot_EL)
        cs_use = brentq(compute_mdot,lower_bound,1e12,args=(REUV,Mp,sigma_EUV,Mdot_EL))
    
    return cs_use


In [ ]:
def compare_densities_EL(REUV,rho_photo,r_p,Mp,sigma_EUV,eff):
    
    rho_EUV = rho_photo * np.exp((G*Mp/cs_eq**2.)*(1./REUV-1/r_p))
    
    cs_use = compute_sound_speed(REUV,Mp,sigma_EUV,eff)
    
    if (cs_use > 1.2e6):
        ## as flow temperature cannot exceed 1e4 K
        cs_use = 1.2e6
    
    Mdot = compute_mdot(cs_use,REUV,Mp,sigma_EUV,0.)
    
    ### now compare to RL flow
    #cs_RL = 1.2e6
    #H = min(REUV/3,cs_RL**2.*REUV**2./(2.*G*Mp))
    #nb = (FEUV/(20*1.6e-12)/(2.6e-13*H))**(1./2.)
    #Rs_RL = G * Mp / (2.*cs_RL**2.)
    #if (REUV <= Rs_RL):
    #    u_RL = FS.get_parker_wind_single(REUV,cs_RL,Rs_RL)
    #else:
    #    u_RL = cs_RL
    
    #Mdot_RL = 4*np.pi*REUV**2.*mh*nb*u_RL
    
    #if (Mdot > Mdot_RL):
        
        ## actually in recombination limit
       
        #Mdot = Mdot_RL
        #cs_use = cs_RL
        

    Rs = G*Mp/2./cs_use**2.
    
    if (REUV <= Rs):
        u_launch = FS.get_parker_wind_single(REUV,cs_use,Rs)
        
    else:
        u_launch = cs_use #assuming launching at the sound-speed
    
    rho_flow = Mdot/(4*np.pi*r_p**2.*u_launch)
    
    #print (r_p,Mdot/1e10,cs_use/1e6)
    
    return (rho_EUV*cs_eq**2.-rho_flow*(u_launch**2.+cs_use**2.))/(2.*rho_EUV*cs_eq**2.)

def compare_densities_RL(REUV,rho_photo,r_p,Mp,sigma_EUV,eff):
    
    rho_EUV = rho_photo * np.exp((G*Mp/cs_eq**2.)*(1./REUV-1/r_p))
    
    #cs_use = compute_sound_speed(REUV,Mp,sigma_EUV,eff)
    
    #if (cs_use > 1.2e6):
    #    cs_use = 1.2e6
    
    #Mdot = compute_mdot(cs_use,REUV,Mp,sigma_EUV,0.)
    
    ### now compare to RL flow
    cs_RL = 1.2e6
    H = min(REUV/3,cs_RL**2.*REUV**2./(2.*G*Mp))
    nb = (FEUV/(20*1.6e-12)/(2.6e-13*H))**(1./2.)
    Rs_RL = G * Mp / (2.*cs_RL**2.)
    if (REUV <= Rs_RL):
        u_RL = FS.get_parker_wind_single(REUV,cs_RL,Rs_RL)
    else:
        u_RL = cs_RL
    
    Mdot_RL = 4*np.pi*REUV**2.*mh*nb*u_RL
    
    #if (Mdot > Mdot_RL):
        
        ## actually in recombination limit
       
    #    Mdot = Mdot_RL
    #    cs_use = cs_RL
        

    Rs = G*Mp/2./cs_RL**2.
    
    if (REUV <= Rs):
        u_launch = FS.get_parker_wind_single(REUV,cs_RL,Rs)
        
    else:
        u_launch = cs_RL #assuming launching at the sound-speed
    
    rho_flow = Mdot_RL/(4*np.pi*r_p**2.*u_launch)
    
    #print (r_p,Mdot/1e10,cs_use/1e6)
    
    return (rho_EUV*cs_eq**2.-rho_flow*(u_launch**2.+cs_RL**2.))/(2.*rho_EUV*cs_eq**2.)
    
def get_Mach_bol_RL(REUV,rho_photo,r_p,Mp,sigma_EUV,eff):
    
    rho_EUV = rho_photo * np.exp((G*Mp/cs_eq**2.)*(1./REUV-1/r_p))
    
    #cs_use = compute_sound_speed(REUV,Mp,sigma_EUV,eff)
    
    #if (cs_use > 1.2e6):
    #    cs_use = 1.2e6
    
    #Mdot = compute_mdot(cs_use,REUV,Mp,sigma_EUV,0.)
    
    ### now compare to RL flow
    cs_RL = 1.2e6
    H = min(REUV/3,cs_RL**2.*REUV**2./(2.*G*Mp))
    nb = (FEUV/(20*1.6e-12)/(2.6e-13*H))**(1./2.)
    Rs_RL = G * Mp / (2.*cs_RL**2.)
    if (REUV <= Rs_RL):
        u_RL = FS.get_parker_wind_single(REUV,cs_RL,Rs_RL)
    else:
        u_RL = cs_RL
    
    Mdot_RL = 4*np.pi*REUV**2.*mh*nb*u_RL
    
    u_bol_RL = Mdot_RL/(4.*np.pi*REUV**2.*rho_EUV)
    
    return Mdot_RL

def get_Mach_bol_EL(REUV,rho_photo,r_p,Mp,sigma_EUV,eff):
    
    rho_EUV = rho_photo * np.exp((G*Mp/cs_eq**2.)*(1./REUV-1/r_p))
    
    cs_use = compute_sound_speed(REUV,Mp,sigma_EUV,eff)
    
    #if (cs_use > 1.2e6):
    #    cs_use = 1.2e6
    
    Mdot = compute_mdot(cs_use,REUV,Mp,sigma_EUV,0.)
    
    u_bol_RL = Mdot / (4*np.pi*REUV**2.*rho_EUV)
    
    return Mdot


def hydro_static_profile(x,b,M_p,cs_eq,r_p,Rlarge):
    
    r = np.sqrt(x**2.+b**2.)
    inside_exp = G*M_p/(r*cs_eq**2.)-G*M_p/(r_p*cs_eq**2.)
    
    rho_profile = np.exp(inside_exp)+1e-100
    
    return rho_profile
    

def compute_transit_radius(r_p,M_p,Rlarge,cs_eq,kappa_p,kappa_transit):
    
    ### this subroutine computes the transit radius
    g = G * M_p / r_p**2.
    rho_photo = g / (kappa_p * cs_eq**2.)
    
    impact_param = np.logspace(np.log10(r_p),np.log10(Rlarge),100)
    tau = np.zeros(np.size(impact_param))
    for i in range(np.size(tau)):
        tau[i] = quad(hydro_static_profile,-Rlarge,Rlarge,args=(impact_param[i],M_p,cs_eq,r_p,Rlarge))[0]
        tau[i]*= rho_photo*kappa_transit
    
    flux_ratio = 2.*np.trapz(impact_param*np.exp(-tau),impact_param)/Rlarge**2.
    
    r_transit = Rlarge*np.sqrt(1.-flux_ratio)
    
    return r_transit
    

In [ ]:
Mp = 232.072*5.97e27
sigma_EUV = 1.89e-18
eff = 0.3

REUV = np.linspace(1e10,1e10,2)
cs_out = np.zeros(np.size(REUV))

for i in range(np.size(cs_out)):

    cs_out[i]=(compute_sound_speed(REUV[i],Mp,sigma_EUV,eff))
    
plt.loglog(REUV,cs_out,'.')


print (cs_out[i])

In [ ]:
Mp_grid = np.logspace(np.log10(232.072),np.log10(1000),1)*5.97e27

sigma_EUV = 1.89e-18
eff = 0.1

Rtau = np.zeros(np.size(Mp_grid))
Rtran = np.zeros(np.size(Mp_grid))
Renh_2 = np.zeros(np.size(Mp_grid))
Renh_1p5 = np.zeros(np.size(Mp_grid))
REL_vs_RRL = np.zeros(np.size(Mp_grid))
Rrav = np.zeros(np.size(Mp_grid))

for j in range (0,np.size(Mp_grid)):
    store_EL_vs_RL = True
    Mp = Mp_grid[j]

    r_p = np.logspace(np.log10(9.9374e+9),np.log10(1e10),2)
    r_t = np.zeros(np.size(r_p))
    rmax_guess = 1.
    REUV = np.zeros(np.size(r_p))
    REUV_EL = np.zeros(np.size(r_p))
    Mach_EL = np.zeros(np.size(r_p))
    Mdot_EL = np.zeros(np.size(r_p))
    Mdot_max_EL = np.zeros(np.size(r_p))
    time_scale_flow = np.zeros(np.size(r_p))
    time_scale_recom = np.zeros(np.size(r_p))
    time_scale_ratio = np.zeros(np.size(r_p)) + 10.
    cs_out = np.zeros(np.size(r_p))
    rmin_guess = r_p[0]

    Rs_iso = G * Mp /2./cs_eq**2.

    for i in range(np.size(r_p)):
        r_t[i] = compute_transit_radius(r_p[i],Mp,2.*Rs_iso,cs_eq,kappa_p,kappa_op)
        
        g = G * Mp / r_p[i]**2.
        rho_photo = g / (kappa_p * cs_eq**2.)
        rmax_guess = 1.
        a=1.
        b=1.
        while (a*b>=0.):
            rmax_guess = rmax_guess * 1.01
            a = compare_densities_EL(rmin_guess,rho_photo,r_p[i],Mp,sigma_EUV,eff)
            b = compare_densities_EL(rmax_guess*r_p[i],rho_photo,r_p[i],Mp,sigma_EUV,eff)
        REUV_EL[i] = brentq(compare_densities_EL,rmin_guess,rmax_guess*r_p[i],args=(rho_photo,r_p[i],Mp,sigma_EUV,eff))
        rmin_guess = REUV_EL[i]
        Mdot_EL[i] = get_Mach_bol_EL(REUV_EL[i],rho_photo,r_p[i],Mp,sigma_EUV,eff)
        rho_EUV = rho_photo * np.exp((G*Mp/cs_eq**2.)*(1./REUV_RL[i]-1/r_p[i]))
        print ()
         
    
        if (REUV_EL[i] > 60.*Rs_iso):
            IMAX_PLOT = i
            break
            
        # compute timescale ratio
        cs_use = compute_sound_speed(REUV_EL[i],Mp,sigma_EUV,eff)
        if (cs_use > 1.2e6):
            ## as flow temperature cannot exceed 1e4 K
            cs_use = 1.2e6
        
        Mach_EL[i] = Mdot_EL[i]/(4.*np.pi*REUV_EL[i]**2.*cs_use * rho_EUV)
        
        cs_out[i] = cs_use
        
        Rs_EL = G*Mp/2./cs_use**2.
    
        if (REUV_EL[i] <= Rs_EL):
            u_launch = FS.get_parker_wind_single(REUV_EL[i],cs_use,Rs_EL)
            u_launch_for_grad = FS.get_parker_wind_single(1.001*REUV_EL[i],cs_use,Rs_EL)
            Hflow = 0.001*REUV_EL[i]/(np.log(1./REUV_EL[i]**2./u_launch)-np.log(1./(1.001*REUV_EL[i])**2./u_launch_for_grad))
        else:
            u_launch = cs_use #assuming launching at the sound-speed
            Hflow = REUV_EL[i]
        
        #Hflow = 1./()
        #print (Hflow)
        
        time_scale_flow[i] = Hflow/u_launch
        time_scale_recom[i] = np.sqrt(Hflow/(FEUV_photon*2.6e-13))
        time_scale_ratio[i] = time_scale_recom[i] / time_scale_flow[i]
        if (cs_use < 1.2e6):
            time_scale_ratio[i] = 10.
            
        rho_EUV = rho_photo * np.exp((G*Mp/cs_eq**2.)*(1./REUV_EL[i]-1/r_p[i]))
        Mdot_max_EL[i] = 4. * np.pi* REUV_EL[i]**2. * rho_EUV * (0.5*cs_eq/cs_out[i]) * cs_eq
            
    #plt.loglog(r_p,REUV_EL/r_p)
    
    REUV_RL = np.zeros(np.size(r_p))
    Mach_RL = np.zeros(np.size(r_p))
    Mdot_max_RL = np.zeros(np.size(r_p))
    rmin_guess = r_p[0]
    
    for i in range(np.size(r_p)):
        g = G * Mp / r_p[i]**2.
        rho_photo = g / (kappa_p * cs_eq**2.)
        rmax_guess = 1.
        a=1.
        b=1.
        while (a*b>=0.):
            rmax_guess = rmax_guess * 1.01
            a = compare_densities_RL(rmin_guess,rho_photo,r_p[i],Mp,sigma_EUV,eff)
            b = compare_densities_RL(rmax_guess*r_p[i],rho_photo,r_p[i],Mp,sigma_EUV,eff)
        REUV_RL[i] = brentq(compare_densities_RL,rmin_guess,rmax_guess*r_p[i],args=(rho_photo,r_p[i],Mp,sigma_EUV,eff))
        rmin_guess = REUV_RL[i]
        Mach_RL[i] = get_Mach_bol_RL(REUV_RL[i],rho_photo,r_p[i],Mp,sigma_EUV,eff)
        rho_EUV = rho_photo * np.exp((G*Mp/cs_eq**2.)*(1./REUV_RL[i]-1/r_p[i]))
        Mdot_max_RL[i] = 4. * np.pi* REUV_RL[i]**2. * rho_EUV * (0.5*cs_eq/1.2e6) * cs_eq
    
        if (REUV_RL[i] > 30.*Rs_iso):
            break
    #plt.loglog(r_p,REUV_RL/r_p,'--')
    
    for i in range(np.size(r_p)):
        REUV[i] = REUV_EL[i]
        if (time_scale_ratio[i] < 1.):
            REUV[i] = -REUV_RL[i]
            if (store_EL_vs_RL):
                REL_vs_RRL[j] = r_t[i]
                store_EL_vs_RL = False
                TRANSITION_INDEX = i
        

    #plt.loglog(Rtau[j],Rs_iso/Rtau[j],'o')
        

In [ ]:
print (cs_out)
print (REUV)
print (REUV/r_p)
print (Mdot_EL)
print (Mach_EL)

In [ ]:
plt.figure(figsize=(4,2.7))

plt.loglog(Mp_grid/mearth,Rtau/rearth,'C1',lw=2,label=r'Photospheric Radius\\ CPML $\leftrightarrow$ PE')
plt.loglog(Mp_grid/mearth,Rtran/rearth,'m-.',lw=2,label=r'Transit Radius\\ CPML $\leftrightarrow$ PE')
plt.legend(loc=0)
plt.loglog(Mp_grid/mearth,Renh_2/rearth,':')
#plt.loglog(Mp_grid/mearth,Rrav/rearth)
#plt.loglog(Mp_grid/5.97e27,1/9*G*Mp_grid/cs_eq**2./2./6.371e8,':')
plt.loglog(Mp_grid/mearth,Renh_1p5/rearth,':')
#plt.legend(loc=0)
plt.loglog(Mp_grid/mearth,REL_vs_RRL/rearth,'C7--')
#plt.loglog(Mp_grid/5.97e27,1/6*G*Mp_grid/cs_eq**2./2./6.371e8,':')
plt.xlim((1.,15.))
plt.ylim((1.,20.))
plt.xlabel(r'Planet Mass [M$_\oplus$]')
plt.ylabel(r'Radius [R$_\oplus$]')
#plt.text(1.13,4.,"Core-powered\n mass-loss")
#plt.text(5.2,2.3,"Photoevaporation")
plt.text(4.3,4.7,"Recombination Limited",rotation=20)
plt.text(3.3,2.9,"Energy Limited",rotation=19)
plt.text(2.15,1.3,r"$R_{XUV}=2R_{\rm p}$",rotation=33)
plt.text(6.4,1.66,r"$R_{XUV}=1.3R_{\rm p}$",rotation=33)
#plt.loglog(Mp_grid/mearth,(Mp_grid/mearth)**0.25)
plt.fill_between(Mp_grid/mearth,np.zeros(np.size(Mp_grid))+1e-10,(Mp_grid/mearth)**0.25,color='brown',alpha=0.3)
plt.loglog(Mp_grid/mearth,G*Mp_grid/(2.*cs_eq**2.)*1./8./rearth,'C7',linestyle=(0, (1, 3)))
plt.loglog(Mp_grid/mearth,G*Mp_grid/(2.*cs_eq**2.)*1./7./rearth,'C7',linestyle=(0, (1, 3)))
plt.loglog(Mp_grid/mearth,G*Mp_grid/(2.*cs_eq**2.)*1./6./rearth,'C7',linestyle=(0, (1, 3)))
plt.loglog(Mp_grid/mearth,G*Mp_grid/(2.*cs_eq**2.)*1./5./rearth,'C7',linestyle=(0, (1, 3)))
plt.xticks([1,10],['1','10'])
plt.yticks([1,10],['1','10'])

plt.savefig("/home/jeowen/Dropbox/Apps/Overleaf/Planetary_Winds_Analytic/Mass_radius3.pdf",bbox_inches='tight')

In [ ]:
print (Rtran)
print (Mp_grid/mearth)

np.save("d1000_1e-4_w_photo",[Rtran,Renh_2,cs_eq,-2,Mp_grid,Rtau],allow_pickle=True)

#plt.loglog(Mp_grid/mearth,Rrav/rearth)

In [ ]:
print (Rtau)

plt.loglog(r_p,REUV_EL)
plt.loglog(r_p,REUV_RL)
plt.loglog(r_p,r_p,'--')

plt.figure()
plt.loglog(r_p,time_scale_flow)
plt.loglog(r_p,time_scale_recom)

plt.figure()
plt.loglog(r_p,cs_out)

plt.figure()
plt.loglog(r_p,time_scale_ratio)

In [ ]:
plt.figure()
plt.loglog(r_p,REUV)
plt.loglog(r_p,REUV_EL,'--')
plt.loglog(r_p,REUV_RL,':')
plt.figure()

plt.loglog(r_p,Mach_EL)
plt.loglog(r_p,Mach_RL)
plt.figure()
rho_EUV1 = rho_photo * np.exp((G*Mp/cs_eq**2.)*(1./REUV_EL-1/r_p))
rho_EUV2 = rho_photo * np.exp((G*Mp/cs_eq**2.)*(1./REUV_RL-1/r_p))

plt.loglog(r_p,rho_EUV1)
plt.loglog(r_p,rho_EUV2)


In [ ]:
print (Rtau)

plt.loglog(Mp_grid/5.97e27,Rtau/6.371e8)
#plt.loglog(Mp_grid/5.97e27,Rtau2/6.371e8,':')
plt.loglog(Mp_grid/5.97e27,(Mp_grid/5.97e27)**0.25,'--')
plt.loglog(Mp_grid/5.97e27,G*Mp_grid/2./cs_eq**2.*(1./8./6.371e8),'-.')

In [ ]:
plt.loglog(r_p/6.371e8,REUV/Rs_iso)
plt.loglog([r_p[0]/6.371e8,r_p[-1]/6.371e8],[1., 1.],'--')
plt.loglog(r_p/6.371e8,7.*r_p/Rs_iso,':')
plt.loglog(r_p/6.371e8,r_p/Rs_iso,'-.')

In [ ]:
ratio = np.linspace(0.5,3)

plt.loglog(ratio,-(1. - 4.*np.log(ratio) - 4./ratio))

In [ ]:
Rs = 1e9

rtest = np.linspace(3.5e9,1e10)

usol = FS.get_parker_wind_const(rtest,1,Rs,-5.)

plt.loglog(rtest,usol)


In [ ]:
print (FS.get_parker_wind_single(0.0703,1.,1.))

In [ ]:
T = np.logspace(3.5,5.)
plt.loglog(T,1.38e-16*T*2.6e-13*(T/1e4)**-0.7/(7.5e-19*np.exp(-118348/T)))

In [ ]:
cr = np.linspace(1.,10.,100)

plt.loglog(cr,cr-np.sqrt(cr**2.-1))
plt.loglog(cr,cr+np.sqrt(cr**2.-1))

In [ ]:
print (0.5*cs_eq/1.2e6)

In [ ]:
print (1.2e6/cs_eq - np.sqrt((1.2e6/cs_eq)**2.-1.))